Semantic search

Reference: https://huggingface.co/learn/nlp-course/chapter5/6?fw=tf

Dataset: https://www.kaggle.com/datasets/arplusman/papers-by-subject

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/papers-by-subject/arXiv-DataFrame.csv


In [2]:
!pip install datasets evaluate transformers[sentencepiece]
!pip install faiss-gpu
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 12.2 MB/s eta 0:00:00


In [3]:
df = pd.read_csv('/kaggle/input/papers-by-subject/arXiv-DataFrame.csv',
                usecols=['Title', 'Summary'])
df = df[:537]
df.head()

,Title,Summary
0,Dynamic Backtracking,Because of their occasional need to return to ...
1,A Market-Oriented Programming Environment and ...,Market price systems constitute a well-underst...
2,An Empirical Analysis of Search in GSAT,We describe an extensive study of search in GS...
3,The Difficulties of Learning Logic Programs wi...,As real logic programmers normally use cut (!)...
4,Software Agents: Completing Patterns and Const...,To support the goal of allowing users to recor...


In [4]:
df.shape

(537, 2)

In [5]:
from datasets import Dataset

ds = Dataset.from_pandas(df)
ds

Dataset({
    features: ['Title', 'Summary'],
    num_rows: 537
})

In [6]:
def concatenate_text(examples):
    return {
        "text": examples["Title"] + " \n " + examples["Summary"]
    }


ds_concat = ds.map(concatenate_text)
ds_concat

  0%|          | 0/537 [00:00<?, ?ex/s]

Dataset({
    features: ['Title', 'Summary', 'text'],
    num_rows: 537
})

### Create embeddings

In [7]:
from transformers import AutoTokenizer, TFAutoModel

model_ckpt = "sentence-transformers/multi-qa-mpnet-base-dot-v1"
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)
model = TFAutoModel.from_pretrained(model_ckpt, from_pt=True)
# from_pt=True will automatically convert PyTorch weights to the TensorFlow format

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: l

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFMPNetModel: ['embeddings.position_ids']
- This IS expected if you are initializing TFMPNetModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFMPNetModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFMPNetModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFMPNetModel for predictions without further training.


In [8]:
def cls_pooling(model_output):
    return model_output.last_hidden_state[:, 0]

In [9]:
def get_embeddings(text_list):
    encoded_input = tokenizer(
        text_list, padding=True, truncation=True, return_tensors="tf"
    )
    encoded_input = {k: v for k, v in encoded_input.items()}
    model_output = model(**encoded_input)
    return cls_pooling(model_output)

In [10]:
embedding = get_embeddings(ds_concat["text"][0])
embedding.shape

TensorShape([1, 768])

In [11]:
embeddings_dataset = ds_concat.map(
    lambda x: {"embeddings": get_embeddings(x["text"]).numpy()[0]}
)

  0%|          | 0/537 [00:00<?, ?ex/s]

In [12]:
embeddings_dataset.add_faiss_index(column="embeddings")

  0%|          | 0/1 [00:00<?, ?it/s]

Dataset({
    features: ['Title', 'Summary', 'text', 'embeddings'],
    num_rows: 537
})

In [13]:
# https://huggingface.co/docs/datasets/access
embeddings_dataset[0]['text']

'Dynamic Backtracking \n Because of their occasional need to return to shallow points in a search tree, existing backtracking methods can sometimes erase meaningful progress toward solving a search problem. In this paper, we present a method by which backtrack points can be moved deeper in the search space, thereby avoiding this difficulty. The technique developed is a variant of dependency-directed backtracking that uses only polynomial space while still providing useful control information and retaining the completeness guarantees provided by earlier approaches.'

In [14]:
question = "How does Dynamic Backtracking improve upon traditional backtracking methods?"
question_embedding = get_embeddings([question]).numpy()
question_embedding.shape

(1, 768)

In [15]:
scores, samples = embeddings_dataset.get_nearest_examples(
    "embeddings", question_embedding, k=5
)

In [16]:
import pandas as pd

samples_df = pd.DataFrame.from_dict(samples)
samples_df["scores"] = scores
samples_df.sort_values("scores", ascending=False, inplace=True)

In [17]:
for _, row in samples_df.iterrows():
    print(f"TITLE: {row.Title}")
    print(f"Summary: {row.Summary}")
    print(f"SCORE: {row.scores}")
    print("=" * 50)
    print()

TITLE: Quantum Computing and Phase Transitions in Combinatorial Search
Summary: We introduce an algorithm for combinatorial search on quantum computers that is capable of significantly concentrating amplitude into solutions for some NP search problems, on average. This is done by exploiting the same aspects of problem structure as used by classical backtrack methods to avoid unproductive search choices. This quantum algorithm is much more likely to find solutions than the simple direct use of quantum parallelism. Furthermore, empirical evaluation on small problems shows this quantum algorithm displays the same phase transition behavior, and at the same location, as seen in many previously studied classical search methods. Specifically, difficult problem instances are concentrated near the abrupt change from underconstrained to overconstrained problems.
SCORE: 46.565040588378906

TITLE: Iterative Deepening Branch and Bound
Summary: In tree search problem the best-first search algorithm 